In [1]:
import os
import sys
import tensorflow as tf
import tensorflow_io as tfio
import numpy as np
from IPython.display import Audio
import matplotlib.pyplot as plt
from tensorflow.python.platform import gfile
from tensorflow.python.summary.writer.writer import FileWriter
from tensorflow.core.framework import summary_pb2

import matplotlib.pyplot as plt
import wuw.data
import wuw.models
import yaml
import json
from tqdm.notebook import trange, tqdm
from PIL import Image

tf.compat.v1.disable_eager_execution()

plt.rcParams["figure.figsize"] = [12, 8]
plt.rcParams["figure.dpi"] = 100

2023-06-02 14:28:55.082204: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-02 14:28:55.945648: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# conf_fn = "conf/lstm-s.yaml"
# expdir = "exp/utt3s_trainset2_lstm_step10k_allbg5"
conf_fn = "conf/dscnn-tiny.yaml"
expdir = "exp/utt3s_trainset2_dscnn-tiny_step10k_allbg5"
logdir = expdir + "/logs"

with open(conf_fn, "r") as stream:
    try:
        conf_yaml = yaml.safe_load(stream)
        # print(json.dumps(conf_yaml, indent=2))
    except yaml.YAMLError as exc:
        print(exc)

In [5]:
tf.compat.v1.reset_default_graph()
sess = tf.compat.v1.InteractiveSession()
model = wuw.models.select_model(
    conf_yaml["data_conf"], conf_yaml["feat_conf"], conf_yaml["model_conf"]
)


conf_yaml["model_conf"] = model.prepare_model_settings()
audio_loader = wuw.data.AudioLoader(
    conf_yaml["data_conf"], conf_yaml["feat_conf"], conf_yaml["model_conf"]
)

2023-06-02 14:30:58.045436: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-02 14:30:58.045764: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-02 14:30:58.045916: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

-----
Set index: validation - Count: dict_items([(<Label.NEGATIVE: 1>, 4974), (<Label.POSITIVE: 0>, 2932)]) - Sum: 7906
Set index: testing - Count: dict_items([(<Label.NEGATIVE: 1>, 1927), (<Label.POSITIVE: 0>, 432)]) - Sum: 2359
Set index: training - Count: dict_items([(<Label.POSITIVE: 0>, 11732), (<Label.NEGATIVE: 1>, 19898)]) - Sum: 31630
Total count: dict_items([(<Label.NEGATIVE: 1>, 26799), (<Label.POSITIVE: 0>, 15096)]) - Sum: 41895
-----
Load background
wav_decoder (48000, 1)
left_position_= Tensor("left_position:0", shape=(), dtype=int32)
scaled_foreground (20800, 1)
background_clamp (20800, 1)
spectrogram_: (1, 64, 513)


2023-06-02 14:31:01.681729: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'left_position' with dtype int32
	 [[{{node left_position}}]]


In [6]:
fingerprint_size = conf_yaml["model_conf"]["fingerprint_size"]
label_count = conf_yaml["model_conf"]["label_count"]
training_steps_list = conf_yaml["model_conf"]["training_steps"]
learning_rates_list = conf_yaml["model_conf"]["learning_rate"]

fingerprint_input = tf.compat.v1.placeholder(
    tf.float32, [None, fingerprint_size], name="fingerprint_input"
)
ground_truth_input = tf.compat.v1.placeholder(
    tf.float32, [None, label_count], name="groundtruth_input"
)
logits, dropout_prob = model.forward(
    fingerprint_input, conf_yaml["model_conf"]["model_size_info"]
)

# Create the back propagation and training evaluation machinery in the graph.
with tf.name_scope("cross_entropy"):
    cross_entropy_mean = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            labels=ground_truth_input, logits=logits
        )
    )
tf.compat.v1.summary.scalar("cross_entropy", cross_entropy_mean)

update_ops = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.UPDATE_OPS)
with tf.name_scope("train"), tf.control_dependencies(update_ops):
    learning_rate_input = tf.compat.v1.placeholder(
        tf.float32, [], name="learning_rate_input"
    )
    train_step = tf.compat.v1.train.AdamOptimizer(learning_rate_input).minimize(
        cross_entropy_mean
    )

predicted_indices = tf.argmax(logits, 1)
expected_indices = tf.argmax(ground_truth_input, 1)
correct_prediction = tf.equal(predicted_indices, expected_indices)
confusion_matrix = tf.math.confusion_matrix(
    expected_indices, predicted_indices, num_classes=label_count
)
evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.compat.v1.summary.scalar("accuracy", evaluation_step)

global_step = tf.compat.v1.train.get_or_create_global_step()
increment_global_step = tf.compat.v1.assign(global_step, global_step + 1)
saver = tf.compat.v1.train.Saver(tf.compat.v1.global_variables())

# Merge all the summaries and write them out to /tmp/retrain_logs (by default)
merged_summaries = tf.compat.v1.summary.merge_all()
train_writer = tf.compat.v1.summary.FileWriter(
    os.path.join(expdir, "train"), sess.graph
)
validation_writer = tf.compat.v1.summary.FileWriter(os.path.join(expdir, "val"))

tf.compat.v1.global_variables_initializer().run()
params = tf.compat.v1.trainable_variables()
num_params = sum(map(lambda t: np.prod(tf.shape(t.value()).eval()), params))
print("Total number of Parameters: {}\n-----".format(num_params))

# Save graph.pbtxt.
# tf.io.write_graph(sess.graph_def, expdir, '{}.pbtxt'.format(conf_yaml["model_conf"]["name"]))
model.load_variables_from_checkpoint(
    sess, tf.compat.v1.train.latest_checkpoint(expdir + "/best")
)

/disk2/softs/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Total number of Parameters: 251438
-----
INFO:tensorflow:Restoring parameters from exp/utt3s_trainset2_dscnn-tiny_step10k_allbg5/best/ds_cnn_9714.ckpt-9699


In [24]:
predicted_output = []

train_size = audio_loader.size("training")
print(f"set_size={train_size}")
total_accuracy = 0
total_conf_matrix = None
for i in trange(0, train_size, conf_yaml["model_conf"]["batch_size"]):
    test_fingerprints, test_ground_truth = audio_loader.load_batch(
        sess,
        conf_yaml["model_conf"]["batch_size"],
        offset=i,
        background_frequency=0,
        background_volume_range=0,
        time_shift=0,
        mode="training",
    )
    test_summary, test_accuracy, test_matrix, correct_prediction_i = sess.run(
        [merged_summaries, evaluation_step, confusion_matrix, correct_prediction],
        feed_dict={
            fingerprint_input: test_fingerprints,
            ground_truth_input: test_ground_truth,
            dropout_prob: 1.0,
        },
    )
    batch_size = min(conf_yaml["model_conf"]["batch_size"], train_size - i)
    total_accuracy += (test_accuracy * batch_size) / train_size
    if total_conf_matrix is None:
        total_conf_matrix = test_matrix
    else:
        total_conf_matrix += test_matrix

    predicted_output_i = audio_loader.data_index["training"][i : i + batch_size]

    predicted_output += [
        predicted_output_i[x] for x in range(batch_size) if not correct_prediction_i[x]
    ]

print(f"Final confusion matrix: \n {total_conf_matrix}")
print(f"Final accuracy {total_accuracy}")

set_size=31630


  0%|          | 0/159 [00:00<?, ?it/s]

Final confusion matrix: 
 [[11482   250]
 [  238 19660]]
Final accuracy 0.9845716212506618


In [40]:
double_check_list_fn = "data/trainsets/trainset2_ftel_clone1_devsil/double_check.list"

label_names = {
    wuw.data.Label.POSITIVE: "positive",
    wuw.data.Label.NEGATIVE: "negative",
    wuw.data.Label.SILENCE: "silence",
}

with open(double_check_list_fn, "wt") as fp:
    for x in predicted_output:
        line = (
            label_names[x["label"]]
            + " {:.2f} ".format(x["right_position"] / 16000)
            + x["file"]
            + "\n"
        )
        fp.write(line)

In [39]:
x["label"]

<Label.NEGATIVE: 1>